## Preparing Datasets & Dependencies

In [1]:
# Paths
db_path = "/kaggle/input/metal-bind-predict/metalBindPredict/db/uniprot_data"
scripts_path = "/kaggle/input/metal-bind-predict/metalBindPredict/scripts"

In [2]:
# Dependencies
import os
import pandas as pd
import matplotlib as plt
import numpy as np
import sys
#import gtfparse
import seaborn as sns
#conda install -c bioconda gtfparse

# global settings
sns.set(rc={'figure.figsize':(11.7,8.27)})
os.chdir(db_path)

eco_definition = {'ECO:0000255': "match to sequence model evidence that is used in a manual assertion.",
 'ECO:0000269': "experimental evidence that is used in a manual assertion.", 
 'ECO:0007744': "combinatorial computational and experimental evidence used in manual assertion",
 'ECO:0000250': "sequence similarity evidence used in manual assertion",
 'ECO:0000305': "curator inference used in manual assertion",
 'ECO:0000312': "imported information used in manual assertion",
 'ECO:0000303': "author statement without traceable support used in manual assertion"
}

evidence = {'ECO:0000255': "OTHER",
 'ECO:0000269': "EXPERI", 
 'ECO:0007744': "EXPERI & COMP",
 'ECO:0000250': "SEQ SIMILAR",
 'ECO:0000305': "CURATOR",
 'ECO:0000312': "OTHER",
 'ECO:0000303': "OTHER"
}

## Functions

In [3]:
# function to read fasta files into dataframes
import pandas as pd
def read_fasta(file_name,file_path, columns) :
    from Bio.SeqIO.FastaIO import SimpleFastaParser 
    with open(file_name) as fasta_file:  
        records = [] # create empty list
        for title, sequence in SimpleFastaParser(fasta_file): #SimpleFastaParser Iterate over Fasta records as string tuples. For each record a tuple of two strings is returned, the FASTA title line (without the leading ‘>’ character), and the sequence (with any whitespace removed). 
            record = []
            title_splits=title.split('|') # Data cleaning is needed
            #print(title_splits)
                 
            
            record.append(title_splits[1])  #First values are ID (Append adds element to a list)
            record.append(len(sequence)) #Second values are sequences lengths
            record.append(title_splits[2]) #It converts into one line
            record.append(sequence)#Third values are sequences
            records.append(record)
    return pd.DataFrame(records, columns = columns) #We have created a function that returns a dataframe

# Getting part of the sequence before and after the binding amino acids with an offset of 3 amino acids
def window_3(df,Position,sequence):
    if Position <= 3:
        return sequence[:Position+3]
    if Position >= df['Position'].max()-3:
        return sequence[Position-4:]
    else:
        return sequence[Position-4:Position+3]


# Getting part of the sequence before and after the binding amino acids with an offset of 10 amino acids
def window_10(df,Position,sequence):
    if Position <= 10:
        return sequence[:Position+10]
    if Position >= df['Position'].max()-10:
        return sequence[Position-11:]
    else:
        return sequence[Position-11:Position+10]

#pos_df["window_3"] = pos_df.apply(lambda row: window_3(pos_df,row["Position"],row["sequence"]),axis=1)
#pos_df["window_10"] = pos_df.apply(lambda row: window_10(pos_df,row["Position"],row["sequence"]),axis=1)

# pos dataframe setup function
def pos_df_setup(fasta_file, tsv_file, chebi_ids):
    fasta_file = read_fasta(fasta_file,db_path, columns=["id","seq_length","info", "sequence"])
    tsv_file = pd.read_csv(db_path+'/'+tsv_file, sep='\t', header=0)
    
    # merging tsv and fasta
    pos_df = pd.merge(tsv_file, fasta_file, left_on='Accession', right_on='id')
    
    # function to determine the amino acid that the binding happened at
    pos_df['binding_residue'] = pos_df.apply(lambda row: row.sequence[row.Position-1] , axis=1)
    
    pos_df['eco_definition'] = pos_df.apply(lambda row: eco_definition[row.Evidence], axis=1)
    pos_df['evidence'] = pos_df.apply(lambda row: evidence[row.Evidence], axis=1)

    
    chebi_id_df = pd.read_csv(db_path+'/'+chebi_ids, sep='\t', header=0)
    chebi_id_df = chebi_id_df[['ChEBI-ID','Name']]
    
    pos_df = pd.merge(pos_df, chebi_id_df, left_on='ChEBI-ID', right_on='ChEBI-ID')
    pos_df = pos_df[['Accession','Evidence','ChEBI-ID','Position','Name','binding_residue','eco_definition','sequence','evidence']]
    pos_df = pos_df.rename(columns={"Name": "metal_ion_name"})
    

    return pos_df
    


In [4]:
# loading all pos datasets
pos_train = pos_df_setup("POS_TRAIN.fasta", 'POS_TRAIN.tsv', 'ChEBI-IDs_for_metal_binding.tsv')
pos_train_full = pos_df_setup("POS_TRAIN_FULL.fasta", 'POS_TRAIN_FULL.tsv', 'ChEBI-IDs_for_metal_binding.tsv')
pos_train_lit_based = pos_train_full[pos_train_full['Evidence']=='ECO:0000269'].drop(columns=['Evidence'])
pos_train_comp_based = pos_train_full[pos_train_full['Evidence']=='ECO:0007744'].drop(columns=['Evidence'])
pos_train_both_based = pos_train.drop(columns=['Evidence']); pos_train_both_based = pos_train_both_based[pos_train_both_based.duplicated()]

## Constructing different DB from Full Pos Train 

In [5]:
test_df = pos_train_full.copy()

# adding unique sequence column
new_df = pd.DataFrame(columns=['sequence'])
for i in set(test_df['sequence']):
    new_df = new_df.append({'sequence': i}, ignore_index=True)
    
# adding uniprot ids that correspond to each sequence
temp = pd.DataFrame(test_df.groupby('sequence')['Accession'].apply(list))
temp['Accession'] = temp['Accession'].apply(set)
new_df = pd.merge(temp, new_df, left_on='sequence', right_on='sequence')

# adding column for number of species with the same sequence
new_df['n_species_with_this_br'] = [len(seq) for seq in new_df['Accession']]

# adding chebi-ids column
temp = pd.DataFrame(test_df.groupby('sequence')['ChEBI-ID'].apply(list))
temp['ChEBI-ID'] = temp['ChEBI-ID'].apply(set)
new_df = pd.merge(temp, new_df, left_on='sequence', right_on='sequence')

# adding evidence ids column
temp = pd.DataFrame(test_df.groupby('sequence')['Evidence'].apply(list))
temp['evidence_ids'] = temp['Evidence'].apply(set)
new_df = pd.merge(temp, new_df, left_on='sequence', right_on='sequence')
new_df = new_df.drop(columns= {'Evidence'})

# adding number of evidences column
new_df['n_evidences'] = [len(seq) for seq in new_df['evidence_ids']]

# adding position column that correspond to the same sequence
temp = pd.DataFrame(test_df.groupby('sequence')['Position'].apply(list))
temp['Position'] = temp['Position'].apply(set)
new_df = pd.merge(temp, new_df, left_on='sequence', right_on='sequence')
new_df.head()

# adjusting df
new_df = new_df.rename(columns={'Accession': 'uniprot_ids','ChEBI-ID': 'chebi_id'})
new_df = new_df[['uniprot_ids','n_species_with_this_br', 'chebi_id', 'evidence_ids', 'n_evidences', 'Position', 'sequence']]
new_df.uniprot_ids = new_df.uniprot_ids.apply(list)
new_df.chebi_id = new_df.chebi_id.apply(list)
new_df.evidence_ids = new_df.evidence_ids.apply(list)
new_df.Position = new_df.Position.apply(list)

new_df.head()

,uniprot_ids,n_species_with_this_br,chebi_id,evidence_ids,n_evidences,Position,sequence
0,[O24396],1,[CHEBI:18420],[ECO:0000255],1,"[64, 91]",AAAAAGRGRSFSPAAPAPSSVRLPGRQAPAPAAASALAVEADPAAD...
1,[P80405],1,[CHEBI:18420],[ECO:0000250],1,"[154, 189]",AAAAAVSGAKRSLRAELKQRLRAISAEERLRCQRLLTQKVIAHRQY...
2,[P83473],1,[CHEBI:29108],[ECO:0000250],1,"[18, 11, 15]",AAAELSLVQLESLREVCEQ
3,[P68734],1,"[CHEBI:29105, CHEBI:29108]",[ECO:0000255],1,"[167, 139, 143, 178, 147, 181, 183, 186]",AAATGSGTTLKGATVPLNISYEGGKYVLRDLSKPTGTQIITYDLQN...
4,[A7TUG9],1,[CHEBI:18420],[ECO:0000250],1,"[209, 302]",AAAWMLNGCLQVMDSRTIPANRNADNVDPALQTATHLCFPTRPVRV...


In [6]:
# saving df as csv
os.chdir('/kaggle/working')
new_df.to_csv('new_df.csv')

# Exploring Duplicates

In [7]:
# function for duplicates exploring 
def duplicate_explore(df):
    temp = df.groupby("sequence")["Accession"].agg(list)
    df["Multi_IDs"] = df["sequence"].map(temp[temp.str.len().ge(2)])
    
    print("This dataset contains (%d) records."%(len(df)))
    
    # counting sequences that has more than one unique id
    count = 0
    for i in df["Multi_IDs"]:
        if type(i) is list:
            if len(set(i))==1:
                count+=1
    print("This dataset contains (%d) sequances that has only one unique id but more than one binding site"%(count))

                
    # counting sequences that has more than one unique id
    count = 0
    for i in df["Multi_IDs"]:
        if type(i) is list:
            if len(set(i))>1:
                count+=1
                #print(set(i))
    
    print("This dataset contains (%d) sequences that has more than one unique id."%(count))
    
    # counting sequences that has only one id and only one binding position
    count = 0
    for i in df["Multi_IDs"]:
        if type(i) is float:
            count+=1
    
    print("This dataset contains (%d) sequences that has only one id and only one binding position"%(count))


In [8]:
#duplicate_explore(pos_train_lit_based)

In [9]:
#duplicate_explore(pos_train_both_based)

In [10]:
#duplicate_explore(pos_train_full)